In [1]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader
import torchvision
from torch.utils.tensorboard import SummaryWriter

c:\Users\Xinxie Wu\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: 'Could not find module 'C:\Users\Xinxie Wu\AppData\Local\Programs\Python\Python310\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Get training & testing data

In [2]:
train_data = torchvision.datasets.CIFAR10(root='../data', train=True, transform=torchvision.transforms.ToTensor(), download=True)

100.0%


Extracting ../data\cifar-10-python.tar.gz to ../data


In [3]:
test_data = torchvision.datasets.CIFAR10(root='../data', train=False, transform=torchvision.transforms.ToTensor(), download=True)

Files already downloaded and verified


In [4]:
train_data_size = len(train_data)
test_data_size = len(test_data)
print(f"Training Dataset Size: {train_data_size}")
print(f"Testing Dataset Size: {test_data_size}")

Training Dataset Size: 50000
Testing Dataset Size: 10000


Use DataLoader to load datasets

In [5]:
train_dataloader = DataLoader(train_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

Build Neural Network

In [6]:
class MyNetwork(nn.Module):
    def __init__(self):
        super(MyNetwork, self).__init__()
        self.model1 = nn.Sequential(
            nn.Conv2d(3, 32, 5, 1, padding=2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 32, 5, 1, padding=2),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 5, 1, padding=2),
            nn.MaxPool2d(2),
            nn.Flatten(),
            nn.Linear(1024, 64),
            nn.Linear(64, 10)
        )

    def forward(self, x):
        output = self.model1(x)
        return output

mynetwork = MyNetwork()

Cost Function

In [7]:
loss_fn = nn.CrossEntropyLoss()

Optimizer

In [9]:
learning_rate = 1e-2
optimizer = optim.SGD(mynetwork.parameters(), lr=learning_rate)

Set up training networks' parameters

In [10]:
total_training_step = 0
total_test_step = 0
epoch = 10
writer = SummaryWriter("../logs_train")

Training - Accuracy: argmax()

In [ ]:
for i in range(epoch):
    print(f"------------epoch: {i+1}------------")

    """
    Training
    """
    mynetwork.train()
    for data in train_dataloader:
        imgs, targets = data
        outputs = mynetwork(imgs)
        loss = loss_fn(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_training_step += 1
        if total_training_step % 100 == 0:
            print(f"Traning: {total_training_step}; Loss: {loss.item()}")
            writer.add_scalar("train_loss", loss.item(), total_training_step)
    
    """
    Testing
    """
    mynetwork.eval()
    total_test_loss = 0
    total_acc = 0
    with torch.no_grad():
        for data in test_dataloader:
            imgs, targets = data
            outputs = mynetwork(imgs)
            loss = loss_fn(outputs, targets)
            total_test_loss += loss.item()
            acc = (outputs.argmax(1) == targets).sum()
            total_acc += acc
    print(f"total test loss: {total_test_loss}")
    print(f"total accuracy: {total_acc/test_data_size}")
    writer.add_scalar("test_loss", total_test_loss, total_test_step)
    writer.add_scalar("test_accuracy", total_acc/test_data_size, total_test_step)
    total_test_step += 1

    torch.save(mynetwork, f"mynetwork_{i}.pth")
    # torch.save(mynetwork.state_dict(), f"mynetwork_{i}.pth")

writer.close()

------------epoch: 11------------
Traning: 1; Loss: 2.2799246311187744
Traning: 2; Loss: 2.2992379665374756
Traning: 3; Loss: 2.316944122314453
Traning: 4; Loss: 2.3101553916931152
Traning: 5; Loss: 2.293660879135132
Traning: 6; Loss: 2.30493426322937
Traning: 7; Loss: 2.3003199100494385
Traning: 8; Loss: 2.30916690826416
Traning: 9; Loss: 2.310387372970581
Traning: 10; Loss: 2.305821418762207
Traning: 11; Loss: 2.3012149333953857
Traning: 12; Loss: 2.3175010681152344
Traning: 13; Loss: 2.2893807888031006
Traning: 14; Loss: 2.296213150024414
Traning: 15; Loss: 2.315534830093384
Traning: 16; Loss: 2.3032915592193604
Traning: 17; Loss: 2.305190324783325
Traning: 18; Loss: 2.3016738891601562
Traning: 19; Loss: 2.3112761974334717
Traning: 20; Loss: 2.3090178966522217
Traning: 21; Loss: 2.3039979934692383
Traning: 22; Loss: 2.3153862953186035
Traning: 23; Loss: 2.2937655448913574
Traning: 24; Loss: 2.2984673976898193
Traning: 25; Loss: 2.3024356365203857
Traning: 26; Loss: 2.307808399200439

: 

: 

In [ ]:
outputs = torch.tensor([[0.1, 0.2],
                        [0.05, 0.4]])
print(outputs.argmax(1))